In [93]:
import requests
import json
from pprint import pprint
class ProdSysApi:
    
    def __init__(self, url, user, password):
        self._auth = requests.auth.HTTPBasicAuth(user, password)
        self._url = url
        
    def api_url(self, id=None):
        if id is None:
            return f'{self._url}/json/'
        else:
            return f'{self._url}/{id}/json/'
    
    def list(self):
        response = requests.get(self.api_url(), auth=self._auth)
        data = response.json()
        return data
        
    def get(self, id):
        response = requests.get(self.api_url(id), auth=self._auth)
        data = response.json()
        return data
        
    def post(self, id, data):
        kwargs = {}
        kwargs['auth'] = self._auth
        kwargs['data'] = {'json': json.dumps(data)}
        response = requests.post(self.api_url(id), auth=self._auth, 
        return response

        
api = ProdSysApi(
    user='api',
    password='Avier1gi',
    url = 'http://universitas.no/admin/api/indesign'
)

#saksliste = api.list()
#pprint(saksliste[5])
# pprint(testsak)
response = api.post(id=23215, data={'arbeidstittel': 'awesome', 'produsert': 0})
print(response.request.headers, response.request.body, sep='\n')
print(response.url, response.status_code, response.text)
testsak = api.get(23215)
print(testsak['arbeidstittel'], testsak['version_no'], testsak['produsert'])

{'User-Agent': 'python-requests/2.18.3', 'Accept-Encoding': 'gzip, deflate', 'Accept': '*/*', 'Connection': 'keep-alive', 'Content-Length': '71', 'Content-Type': 'application/x-www-form-urlencoded', 'Authorization': 'Basic YXBpOkF2aWVyMWdp'}
json=%7B%22arbeidstittel%22%3A+%22awesome%22%2C+%22produsert%22%3A+0%7D
http://universitas.no/admin/api/indesign/23215/json/ 200 {"status":true,"nye_bilete":false,"sql":"UPDATE prodsak SET arbeidstittel='awesome', produsert='0' WHERE prodsak_id='23215' and version_no='1'","rows":1}
awesome 1 0


In [109]:
saker = api.list()
til_web = list(filter(lambda sak: sak['produsert'] == '6', saker))
pprint(til_web[7])


{'arbeidstittel': '16 ANMØYA Unge Ferrari',
 'mappe': 'anmeldelser',
 'prodsak_id': '23190',
 'produsert': '6',
 'tekst': '@tit:Best med sine boys\r\n'
          '\r\n'
          '@fak1:Unge Ferrari\r\n'
          '\r\n'
          '@fak2:Dag: Lørdag\r\n'
          '\r\n'
          'Scene: Vindfruen\r\n'
          '\r\n'
          '@txt:En tettpakket folkemengde står som tente lys idet Stig Haugen, '
          'aka Unge Ferrari, tilbakelent entrer scenen ikledd en ferrari-gul '
          'kjøredress og smale solbriller. Han kaster ikke bort tiden, og '
          'hiver seg rett ut i første låt.\r\n'
          '\r\n'
          'Problemet er bare at han lener seg fullstendig på playback. Låtene '
          'snurres i gang som fra Spotify, hvorpå Unge Ferrari kun «hopper på» '
          'etter hvert. Det hele topper seg da han minst to ganger glemmer når '
          'verset begynner, og er for sen med å føre mikrofonen opp til '
          'munnen. Det oppleves som uprofesjonelt\r\n'
      

In [110]:
pprint(api.get(23194))

{'arbeidstittel': 'NETT Øya Broen og Lars Vaular',
 'bilete': [{'bildefil': '16-ANM-broen-HKH-11.jpg',
             'bildetekst': '',
             'kommentar': '',
             'prioritet': '1',
             'prodbilde_id': '51792',
             'prodsak_id': '23194'},
            {'bildefil': '16-ANM-larsvaular-HKH-12.jpg',
             'bildetekst': '',
             'kommentar': '',
             'prioritet': '2',
             'prodbilde_id': '51793',
             'prodsak_id': '23194'}],
 'dato': '2017-08-13 15:15:26',
 'flagg': '4',
 'id': '155997',
 'journalist': 'MG',
 'kommentar': 'Vellykket eksport fra InDesign!  Endring: -99 tegn\n'
              'Tuesday, August 15 2017 15:32:20\n'
              'null',
 'mappe': 'anmeldelser',
 'prodsak_id': '23194',
 'produsert': '6',
 'tekst': '@tit:Eventyrbrua på Vindfruen\r\n'
          '\r\n'
          '@ing:Broen bringer noe friskt og spennende til Øya, men '
          'improviserer seg tidvis bort i kakofonien.\r\n'
          '\r\n'
  

In [97]:
def deserialize_story(story):
    output = [] 
    head_tags = [('tit', 'title'),('ing', 'lede'),('tema', 'theme_word'),('stikktit', 'kicker')]
    for tag, attr in head_tags:
        text = getattr(story, attr)
        if text:
            output.append(f'@{tag}:{text}')
    for bl in story.byline_set.select_related('contributor'):
        output.append(str(bl))
    output.append(story.bodytext_markup) 
    for aside in story.asides():
        output.append(aside.child.bodytext_markup)
    for pullquote in story.pullquotes():
        output.append(pullquote.child.bodytext_markup)
    return '\n'.join(output)
    
# story = Story.objects.order_by('?').last()
txt1 = deserialize_story(story)
prodstory = api.get(story.prodsak_id)
txt2 = prodstory.get('tekst')
print(story.prodsak_id)
print(txt1)
print()
print(txt2)










22361
@tit:INTERVJUET: Ville ikke kle av seg – blir hetset på nett etter TV2-intervju
@ing:Sykepleierstuden Gyda Løken Sirseth ville fritas fra å kle av seg og bli kroppsvasket av medstudenter. Nå angripes hun i flere internettforum.
@tema:Sykepleien
@bl: Tekst: Gard Oterholm
@bl: Foto: Adrian Nielsen
@box:> 1

@spm:– Hetsen startet etter intervjuet med [TV2](1), men hvordan startet egentlig det hele?
– Det begynte med at vi skulle kle av oss til undertøyet og bli vasket av medstudenter. Jeg ble sjokkert, og sa til veilederen at jeg ikke taklet dette. Jeg ble sendt videre til studielederen som fortalte at jeg ikke hadde noe valg. Mot min vilje ble jeg derfor med.

@spm:– For det handlet bare om medstudenter. Du kunne vasket en naken pasient helt fint?
– Hvis folk tror dette handler om å vaske andre har de misforstått kraftig. Jeg vil ikke kle av meg foran medstudenter og la de vaske meg. Jeg har jobbet fem år med helse, blant annet på sykehjem, og har ingen problemer med å vaske andre.

In [ ]:
import difflib
import IPython
differ = difflib.HtmlDiff(wrapcolumn=70)
table = differ.make_table(txt1.splitlines(), txt2.splitlines())
IPython.display.HTML(table)

In [75]:
story.prodsak_id

22385